In [1]:
import itertools

In [4]:
def split_protein_sequence_classII(protein_sequence, min_length=13, max_length=25):
    """
    Splits a protein sequence into peptides suitable for Class II MHC binding, generating most possible overlapping peptides
    between the minimum and maximum length specified.

    Args:
        protein_sequence (str): The amino acid sequence of the protein.
        min_length (int): The minimum length of each peptide (default is 13).
        max_length (int): The maximum length of each peptide (default is 25).

    Returns:
        list: A list of peptide sequences.
    """
    peptides = []
    for length in range(min_length, max_length + 1):
        peptides.extend([protein_sequence[i:i+length] 
                         for i in range(0, len(protein_sequence) - length + 1)])
    return peptides

In [ ]:
# Example usage
protein_sequence = "MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAG"

# Class II peptides
classII_peptides = split_protein_sequence_classII(protein_sequence)
print("Class II Peptides:", classII_peptides)

In [31]:
import requests
import pandas as pd
from io import StringIO
from itertools import product

IEDB_API_URL_CLASSII = "http://tools-cluster-interface.iedb.org/tools_api/mhcii/"


def run_netmhciipan_binding_affinity_classII(peptides, hla_alleles):
    """
    Uses IEDB API to generate binding affinity for each peptide and HLA interaction for Class II MHC.

    Args:
        peptides (list): A list of peptide sequences.
        hla_alleles (list): A list of HLA alleles for which to make predictions.

    Returns:
        dict: The binding affinity results for each peptide and HLA allele combination.
    """
    # Prepare sequence text in FASTA format and escape special characters
    sequence_text = "".join([f">peptide{i}\n{peptide}\n" for i, peptide in enumerate(peptides)])
    hla_alleles_str = ",".join(hla_alleles)

    # Prepare the payload for the POST request
    payload = {
        "method": "netmhciipan-4.1",
        "sequence_text": sequence_text,
        "allele": hla_alleles_str,
        "length": "asis"
    }

    # Make the POST request to IEDB API
    try:
        response = requests.post(IEDB_API_URL_CLASSII, data=payload)
        response.raise_for_status()
        return {"results": response.text}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

In [29]:
# Example usage for Class II
peptides_classII = ["DPTIEDSYRKQVVID", "IQLIQNHFVDEYDPTIEDSYRKQ"]
hla_alleles_classII = ["HLA-DRB1*01:01", "HLA-DQA1*05:01/DQB1*02:01"]
result_classII = run_netmhciipan_binding_affinity_classII(peptides_classII, hla_alleles_classII)
print("Class II Results:", result_classII)

Class II Results: {'results': 'allele\tseq_num\tstart\tend\tlength\tcore_peptide\tpeptide\tic50\trank\nHLA-DQA1*05:01/DQB1*02:01\t2\t1\t23\t23\tYDPTIEDSY\tIQLIQNHFVDEYDPTIEDSYRKQ\t369.97\t3.7\nHLA-DRB1*01:01\t2\t1\t23\t23\tFVDEYDPTI\tIQLIQNHFVDEYDPTIEDSYRKQ\t1370.67\t84.0\nHLA-DQA1*05:01/DQB1*02:01\t1\t1\t15\t15\tIEDSYRKQV\tDPTIEDSYRKQVVID\t10064.76\t86.0\nHLA-DRB1*01:01\t1\t1\t15\t15\tIEDSYRKQV\tDPTIEDSYRKQVVID\t3987.26\t91.0\n'}


In [13]:
import pandas as pd
from io import StringIO

def process_classII_results(results_text):
    """
    Processes the Class II results returned by the IEDB API.

    Args:
        results_text (str): The raw results text returned by the API.

    Returns:
        DataFrame: A pandas DataFrame with processed results.
    """
    # Use StringIO to convert the string into a file-like object
    results_io = StringIO(results_text)
    # Read the results into a pandas DataFrame
    df = pd.read_csv(results_io, sep="\t")
    return df

# Process the results for Class II
if "results" in result_classII:
    df_classII_results = process_classII_results(result_classII["results"])
    print("Processed Class II Results:")
    print(df_classII_results)
else:
    print("Error in Class II Results:", result_classII["error"])

Processed Class II Results:
                      allele  seq_num  start  end  length core_peptide  \
0  HLA-DQA1*05:01/DQB1*02:01        2      1   23      23    YDPTIEDSY   
1             HLA-DRB1*01:01        2      1   23      23    FVDEYDPTI   
2  HLA-DQA1*05:01/DQB1*02:01        1      1   15      15    IEDSYRKQV   
3             HLA-DRB1*01:01        1      1   15      15    IEDSYRKQV   

                   peptide      ic50  rank  
0  IQLIQNHFVDEYDPTIEDSYRKQ    369.97   3.7  
1  IQLIQNHFVDEYDPTIEDSYRKQ   1370.67  84.0  
2          DPTIEDSYRKQVVID  10064.76  86.0  
3          DPTIEDSYRKQVVID   3987.26  91.0  
